Резников Никита Вячеславович; ТФэ-01-21; @reznya_tv

Задание №5  
Оценить расходы в цилиндр высокого давления и в конденсатор турбины без промежуточного переграва пара.

In [1]:
import iapws
from iapws import IAPWS97 as gas

In [2]:
MPa = 10 ** 6
kPa = 10 ** 3
MBt = 10 ** 6
unit = 1 / MPa
to_kelvin = lambda x: x + 273.15 if x else None

In [3]:
electrical_power = 250 * MBt 
p_0 = 23.5 * MPa 
t_0 = 540 
p_k = 6.9 * kPa 
z = 7
t_feed_water = 263
p_feed_water = 1.35 * p_0 
t_nas_0 = 374.2
mechanical_efficiency = 0.992
generator_efficiency = 0.99 
internal_efficiency = 0.85 

In [4]:
delta_cvd = 0.05 * p_0 
real_p_0 = p_0 - delta_cvd
teor_point_0 = gas(P = p_0 * unit, T=to_kelvin(t_0))
point_0 = gas(P=real_p_0 * unit, h=teor_point_0.h)
point_feed_water = gas(P=p_feed_water * unit, T=to_kelvin(t_feed_water))
point_k_kip_water = gas(P=p_k * unit, x=0)
teor_point_k = gas(P=p_k * unit, s = teor_point_0.s)
heat_drop_i = ( teor_point_0.h - teor_point_k.h) * internal_efficiency
h_k = teor_point_0.h - heat_drop_i
point_k = gas(P = p_k * unit, h = h_k)

In [5]:
# Расчет относительного КПД ЦВД
efficiency_oi = (teor_point_0.h - point_k.h)/(teor_point_0.h - teor_point_k.h)
efficiency_oi

0.8500000000000002

In [6]:
numenator_without = point_k.T * (teor_point_0.s - point_k_kip_water.s)
denumenator_without = teor_point_0.h - point_k_kip_water.h
without_part = 1 - (numenator_without/denumenator_without)
numenator_infinity = point_k.T * (teor_point_0.s - point_feed_water.s)
denumenator_infinity =  teor_point_0.h - point_feed_water.h
infinity_part = 1 - (numenator_infinity / denumenator_infinity)
ksi_infinity = 1 - (without_part / infinity_part)
ksi_infinity

0.14709057780648582

In [7]:
koef = (point_feed_water.T - point_k.T) / (to_kelvin(t_nas_0) - point_k.T)
koef

0.668521593152601

In [8]:
ksi = 0.86 * ksi_infinity
ksi

0.1264978969135778

In [9]:
eff_num = (teor_point_0.h - teor_point_k.h) * internal_efficiency
eff_denum =  teor_point_0.h - point_k_kip_water.h
efficiency = (eff_num / eff_denum) * (1 + ksi)
efficiency,eff_num,eff_denum

(0.4256665812252009, 1195.040745623017, 3162.5947303533876)

In [10]:
estimated_heat_drop = efficiency * (teor_point_0.h - point_feed_water.h)
estimated_heat_drop

926.282537747384

In [11]:
inlet_mass_flow = electrical_power / (estimated_heat_drop * 1000 * mechanical_efficiency * generator_efficiency)
inlet_mass_flow

274.8208415072751

In [12]:
mass_flow_denum = 1000 * (point_k.h - point_k_kip_water.h) * mechanical_efficiency * generator_efficiency
mass_flow_factor = (1/efficiency) - 1
condenser_mass_flow = (electrical_power * mass_flow_factor)/mass_flow_denum
condenser_mass_flow

174.56654510053676

In [13]:
print("Массовый расход в турбину на входе", inlet_mass_flow)

Массовый расход в турбину на входе 274.8208415072751


In [14]:
print("Массовый расход в конденсатор:", condenser_mass_flow)

Массовый расход в конденсатор: 174.56654510053676
